In [ ]:
# if you use Google Colab, then you can connect current Notebook with your Google Drive 
# uncomment code line below
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/Kaggle

/content/drive/MyDrive/Kaggle


In [ ]:
import pandas as pd
import numpy as np
import networkx as nx
import pickle
from sklearn.model_selection import cross_val_score
from gensim.models import KeyedVectors

In [ ]:
import re
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ast
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.corpus import stopwords
import sys
import nltk
import tqdm
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

# Data

In [ ]:
import json
import pandas as pd
from sklearn.model_selection import train_test_split

train = pd.read_json('train_set.json')
test = pd.read_json('test_set.json')

In [ ]:
train.info()
train

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4000 entries, 0 to 3999
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      4000 non-null   int64 
 1   text    4000 non-null   object
 2   label   4000 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 93.9+ KB


,id,text,label
0,0,Remains dating to the 5th century were found i...,1
1,1,Professional Identification is a type of socia...,1
2,2,A magistrate has refused a media ban on the tr...,1
3,3,The Conservation Commons is the expression of ...,1
4,4,Myer is holding a 75 per cent of designer labe...,0
...,...,...,...
3995,3995,"Matthew Hall , 25 , climbed onto the balconies...",0
3996,3996,"Peter Barnett , 43 , travelled from Haddenham ...",1
3997,3997,Lewis Hamilton attended Nelson Mandela 's 90th...,1
3998,3998,Cyanea is a genus of flowering plants in the f...,0


real text - 1

In [ ]:
train[train['label'] == 1]['label'].count()

2016

generated text - 0

In [ ]:
train[train['label'] == 0]['label'].count()

1984

In [ ]:
test.info()
test

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4000 entries, 0 to 3999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      4000 non-null   int64 
 1   text    4000 non-null   object
dtypes: int64(1), object(1)
memory usage: 62.6+ KB


,id,text
0,0,David Cameron revealed he and his wife Samanth...
1,1,David Cameron appeared to forget which footbal...
2,2,Ebola tests for a senior doctor has come back ...
3,3,Celebrity chef's Fat Duck named eighth best re...
4,4,"Navinder Singh Sarao , 36 , is accused of maki..."
...,...,...
3995,3995,"Bob Katter , federal MP for Kennedy , is of Le..."
3996,3996,Man came to hospital complaining his thumb hur...
3997,3997,London weighting is an allowance paid to certa...
3998,3998,"The Toronto Signals Band (abbreviated to ""Sigs..."


# Preprocessing

In [ ]:
stop_words = set(stopwords.words('english'))
stop_words

{'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'only',
 'or',
 'other',
 'our',
 'ours',
 'ourselves',
 'out',
 'over',
 'own',
 'r

In [ ]:
def clean_text(text):
    text = text.lower()
    p = re.split("\W+", text)
    p = [l for l in p if l.isalpha() and l not in stop_words]
    # return ' '.join(p)
    return p

# Clean text

In [ ]:
tqdm.tqdm.pandas()

train['clean_text'] = train.progress_apply(lambda x: clean_text(x['text']), axis=1)

100%|██████████| 4000/4000 [00:00<00:00, 13722.87it/s]


In [ ]:
test['clean_text'] = test.progress_apply(lambda x: clean_text(x['text']), axis=1)

100%|██████████| 4000/4000 [00:00<00:00, 13641.51it/s]


In [ ]:
train.head()

,id,text,label,clean_text
0,0,Remains dating to the 5th century were found i...,1,"[remains, dating, century, found, tomb, hwangn..."
1,1,Professional Identification is a type of socia...,1,"[professional, identification, type, social, i..."
2,2,A magistrate has refused a media ban on the tr...,1,"[magistrate, refused, media, ban, trial, sex, ..."
3,3,The Conservation Commons is the expression of ...,1,"[conservation, commons, expression, cooperativ..."
4,4,Myer is holding a 75 per cent of designer labe...,0,"[myer, holding, per, cent, designer, label, sa..."


In [ ]:
train.loc[0, 'text']

"Remains dating to the 5th century were found in tomb in Hwangnam-dong . Show a man 's bones on top of a woman 's who was buried with jewellery . Experts believe Silla Dynasty-era tomb was built for a noblewoman and her lover or bodyguard was sacrificed and buried on top of her . There 's a suggestion that the set-up may have been designed to show two people having sex - and the Silla were known for their explicit pottery ."

In [ ]:
print(train.loc[0, 'clean_text'])

['remains', 'dating', 'century', 'found', 'tomb', 'hwangnam', 'dong', 'show', 'man', 'bones', 'top', 'woman', 'buried', 'jewellery', 'experts', 'believe', 'silla', 'dynasty', 'era', 'tomb', 'built', 'noblewoman', 'lover', 'bodyguard', 'sacrificed', 'buried', 'top', 'suggestion', 'set', 'may', 'designed', 'show', 'two', 'people', 'sex', 'silla', 'known', 'explicit', 'pottery']


In [ ]:
type(train.loc[0, 'text']), type(train.loc[0, 'clean_text'])

(str, list)

# Train emdeddings

In [ ]:
# learns the embeddings of each text
tagged_text_train = [TaggedDocument(d, [i]) for i, d in enumerate(train['clean_text'])]
tagged_text_test = [TaggedDocument(d, [i]) for i, d in enumerate(test['clean_text'])]

In [ ]:
tagged_text_train[0]

TaggedDocument(words=['remains', 'dating', 'century', 'found', 'tomb', 'hwangnam', 'dong', 'show', 'man', 'bones', 'top', 'woman', 'buried', 'jewellery', 'experts', 'believe', 'silla', 'dynasty', 'era', 'tomb', 'built', 'noblewoman', 'lover', 'bodyguard', 'sacrificed', 'buried', 'top', 'suggestion', 'set', 'may', 'designed', 'show', 'two', 'people', 'sex', 'silla', 'known', 'explicit', 'pottery'], tags=[0])

In [ ]:
#train
dm = 1 #method DM

dim = 64 # dimension of the embeddings
model_docs = Doc2Vec(tagged_text_train, vector_size=dim, window=15,
                dm=dm, min_count=2, epochs=150, workers=10)
if dm == 0:
    model_docs.save('doc2vec_docs_dbow')
else:
    model_docs.save('doc2vec_docs_dm')

## Save embeddings

In [ ]:
#loading model 
fname = 'doc2vec_docs_dm'
method = 'dm'
model_docs = Doc2Vec.load(fname)

In [ ]:
train['text_emb'] = train.progress_apply(lambda x: model_docs.infer_vector(x['clean_text']), axis=1)

100%|██████████| 4000/4000 [02:17<00:00, 29.20it/s]


In [ ]:
train.to_csv(path_or_buf='./train_emb_dm.csv', index=False)

In [ ]:
train.head()

,id,text,label,clean_text,text_emb
0,0,Remains dating to the 5th century were found i...,1,"[remains, dating, century, found, tomb, hwangn...","[0.38071018, -0.7525629, 0.7213382, 1.5455517,..."
1,1,Professional Identification is a type of socia...,1,"[professional, identification, type, social, i...","[0.2909179, 1.3623945, 0.0050003873, -1.035217..."
2,2,A magistrate has refused a media ban on the tr...,1,"[magistrate, refused, media, ban, trial, sex, ...","[2.5276241, 0.8235195, 1.0062376, -1.4121058, ..."
3,3,The Conservation Commons is the expression of ...,1,"[conservation, commons, expression, cooperativ...","[-0.60840833, 0.9133234, -1.4918606, 0.0744777..."
4,4,Myer is holding a 75 per cent of designer labe...,0,"[myer, holding, per, cent, designer, label, sa...","[-0.67441446, 0.9799105, 1.0267773, 0.5797521,..."


In [ ]:
test['text_emb'] = test.progress_apply(lambda x: model_docs.infer_vector(x['clean_text']), axis=1)

100%|██████████| 4000/4000 [01:58<00:00, 33.74it/s]


In [ ]:
test.to_csv(path_or_buf='./test_emb_dm.csv', index=False)

In [ ]:
test.head()

,id,text,clean_text,text_emb
0,0,David Cameron revealed he and his wife Samanth...,"[david, cameron, revealed, wife, samantha, fal...","[2.2832088, -1.9718033, 0.514118, -1.334395, -..."
1,1,David Cameron appeared to forget which footbal...,"[david, cameron, appeared, forget, football, t...","[3.5691993, 2.7359586, -1.0264672, -0.81989056..."
2,2,Ebola tests for a senior doctor has come back ...,"[ebola, tests, senior, doctor, come, back, neg...","[2.6414332, -0.6613307, -3.9496205, -0.1249569..."
3,3,Celebrity chef's Fat Duck named eighth best re...,"[celebrity, chef, fat, duck, named, eighth, be...","[-1.3250257, -1.3351707, -3.390739, 0.23472978..."
4,4,"Navinder Singh Sarao , 36 , is accused of maki...","[navinder, singh, sarao, accused, making, ille...","[-2.153574, -0.7063613, 5.152727, 1.7783477, -..."


In [ ]:
type(test.loc[0, 'text_emb'])

numpy.ndarray

In [ ]:
# learns the embeddings of each text
tagged_text_train = [TaggedDocument(d, [i]) for i, d in enumerate(train['clean_text'])]
tagged_text_test = [TaggedDocument(d, [i]) for i, d in enumerate(test['clean_text'])]

In [ ]:
len(tagged_text_test)

4000

# Classifier

In [ ]:
import xgboost as xgb
import lightgbm as lgb
from sklearn.linear_model import LogisticRegression

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score

In [ ]:
import json
import pandas as pd
from sklearn.model_selection import train_test_split

train = pd.read_csv('train_emb_dm.csv')
test = pd.read_csv('test_emb_dm.csv')

In [ ]:
train.head()

,id,text,label,clean_text,text_emb
0,0,Remains dating to the 5th century were found i...,1,"['remains', 'dating', 'century', 'found', 'tom...",[ 3.8071018e-01 -7.5256288e-01 7.2133821e-01 ...
1,1,Professional Identification is a type of socia...,1,"['professional', 'identification', 'type', 'so...",[ 2.9091790e-01 1.3623945e+00 5.0003873e-03 ...
2,2,A magistrate has refused a media ban on the tr...,1,"['magistrate', 'refused', 'media', 'ban', 'tri...",[ 2.5276241 0.8235195 1.0062376 -1.412105...
3,3,The Conservation Commons is the expression of ...,1,"['conservation', 'commons', 'expression', 'coo...",[-6.0840833e-01 9.1332340e-01 -1.4918606e+00 ...
4,4,Myer is holding a 75 per cent of designer labe...,0,"['myer', 'holding', 'per', 'cent', 'designer',...",[-0.67441446 0.9799105 1.0267773 0.579752...


In [ ]:
X_doc2vec_dm = []
for c in train['text_emb'].values:
    X_doc2vec_dm.append(list(c))

X_train, X_test, y_train, y_test = train_test_split(
    np.array(X_doc2vec_dm), train['label'].values, test_size=0.33, random_state=42)

In [ ]:
clf = xgb.XGBClassifier()

clf.fit(X_train, y_train)
preds = clf.predict(X_test)
accuracy_score(preds, y_test)

0.5772727272727273

In [ ]:
clf = xgb.XGBClassifier(booster='gblinear')

clf.fit(X_train, y_train)
preds = clf.predict(X_test)
accuracy_score(preds, y_test)

0.5621212121212121

In [ ]:
clf = lgb.LGBMClassifier()

clf.fit(X_train, y_train)
preds = clf.predict(X_test)
accuracy_score(preds, y_test)

0.5893939393939394

In [ ]:
clf = LogisticRegression()

clf.fit(X_train, y_train)
preds = clf.predict(X_test)
accuracy_score(preds, y_test)

0.5606060606060606

#  sub1

In [ ]:
def convert_text_doc2vec_dm(T_train, DM_train, T_test, DM_test):
    TF_train = vectorizer.fit_transform(T_train)
    TF_test = vectorizer.transform(T_test)
    TF_train = TF_train.toarray()
    TF_test = TF_test.toarray()

    DM_train = np.array([list(c) for c in DM_train.values])
    DM_test = np.array([list(c) for c in DM_test.values])

    X_train = np.concatenate((TF_train, DM_train), axis=1)
    X_test = np.concatenate((TF_test, DM_test), axis=1)
    return X_train, X_test

In [ ]:
X_train, X_test = convert_text_doc2vec_dm(train['text'], train['text_emb'],
                                          test['text'], test['text_emb'])

In [ ]:
print(X_train.shape, X_test.shape)

(4000, 10064) (4000, 10064)


In [ ]:
from sklearn.model_selection import KFold, GridSearchCV

cvK = KFold(n_splits=5, shuffle=True, random_state=42)

In [ ]:
params_set = {
    'n_estimators' : [100, 300, 500,]
}

lgbm = lgb.LGBMClassifier(random_state=42)

gsearch0 = GridSearchCV(estimator= lgbm, param_grid= params_set, 
                        scoring= 'accuracy', cv= cvK,
                        n_jobs=-1)

gsearch0.fit(X_train, train['label'])
pd.DataFrame(gsearch0.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,12.000012,2.808475,0.063961,0.014744,100,{'n_estimators': 100},0.68125,0.66625,0.63875,0.65125,0.6475,0.65700,0.015033,3
1,29.486571,1.838615,0.088673,0.028143,300,{'n_estimators': 300},0.68750,0.67875,0.64375,0.65250,0.6700,0.66650,0.016229,2
2,53.718660,5.085073,0.137776,0.042856,500,{'n_estimators': 500},0.68750,0.66375,0.65375,0.67125,0.6750,0.67025,0.011275,1


In [ ]:
params_set = {
    'n_estimators' : [750]
}

lgbm = lgb.LGBMClassifier(random_state=42)

gsearch1 = GridSearchCV(estimator= lgbm, param_grid= params_set, 
                        scoring= 'accuracy', cv= cvK,
                        n_jobs=-1)

gsearch1.fit(X_train, train['label'])
pd.DataFrame(gsearch1.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,55.276268,9.502093,0.151021,0.007867,750,{'n_estimators': 750},0.68625,0.66625,0.655,0.6675,0.67,0.669,0.010044,1


In [ ]:
clf = lgb.LGBMClassifier(random_state=42, n_estimators=500)
clf.fit(X_train, train['label'])
predictions = clf.predict(X_test)

In [ ]:
predictions.shape

(4000,)

In [ ]:
sum(predictions)

3595

In [ ]:
clf = lgb.LGBMClassifier(random_state=42)
clf.fit(X_train, train['label'])
predictions = clf.predict(X_test)
sum(predictions)


3608

In [ ]:
import csv

# Write predictions to a file
with open("sub1.csv", "w") as pred:
    csv_out = csv.writer(pred)
    csv_out.writerow(['id','label'])
    for i, row in enumerate(predictions):
        csv_out.writerow([i, row])